## Terry Traffic Stops Prediction for Supreme Court 

###  Business Understanding

#### Overview
Arrest decisions following Terry Stops are critical moments in policing, with significant legal, social, and ethical implications. While such stops are based on "reasonable suspicion," concerns have been raised about potential disparities in arrest outcomes across race and gender. In this project, I aim to use historical stop data to predict whether an arrest is likely to occur after a Terry Stop. By identifying key patterns and risk factors, law enforcement agencies can gain deeper insights into their decision-making processes, assess potential biases, and take data-informed steps toward fairer and more accountable policing practices.

#### Business Problem

Discretionary arrest decisions during Terry Stops have drawn increasing scrutiny due to concerns about potential bias and lack of transparency. Law enforcement agencies face challenges in understanding the key factors influencing whether an arrest is made, particularly when outcomes may disproportionately affect certain demographic groups. Without data-driven insights, it becomes difficult to ensure fair and accountable policing practices.

To address this, I aim to:

1. Predict the likelihood of an arrest following a Terry Stop.

2. Identify the most influential factors contributing to arrest decisions.

3. Provide actionable insights to help law enforcement evaluate patterns, detect disparities, and guide equitable policing interventions.

## Data 

### Data Understanding

The dataset being used for this project was obtained from City of Seattle Open data Portal. Here, I am going to review the dataset to assess the structure and characteristics of the data.


1. Loading Data Set

In [1]:
# Importing Relevant Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE

In [2]:
# Loading Data
df= pd.read_csv('Terry_Stops_20250719.csv')

In [3]:
# Display the first 5 rows
df.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,46 - 55,-1,2.018000e+13,481899,Field Contact,None,8544,1993,Female,Hispanic,...,47:00.0,-,-,-,NORTH PCT 2ND WATCH - B/N RELIEF,N,N,West,Q,Q2
1,18 - 25,-1,2.018000e+13,445585,Offense Report,None,8588,1986,Female,White,...,43:00.0,"OBS WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/D...","WEAPON, PERSON WITH - GUN",911,TRAINING - FIELD TRAINING SQUAD,N,Y,North,J,J1
2,26 - 35,-1,2.017000e+13,234548,Offense Report,None,4852,1953,Male,Asian,...,36:00.0,DISTURBANCE,THEFT - SHOPLIFT,911,NORTH PCT 2ND W - LINCOLN - PLATOON 1,N,N,North,L,L3
3,36 - 45,49326761681,2.023000e+13,49327076666,Field Contact,Knife/Cutting/Stabbing Instrument,7766,1984,Male,White,...,25:58.0,OBS - DOWN - CHECK FOR PERSON DOWN,SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,WEST PCT 1ST W - QUEEN (DAVID) - PLATOON 1,N,Y,West,K,K3
4,Jan-17,48319519002,2.023000e+13,48319446887,Field Contact,-,8765,1992,Male,Two or More Races,...,53:15.0,MVC - HIT AND RUN,TRAFFIC - MV COLLISION INVESTIGATION,911,SOUTH PCT 3RD W - ROBERT (OCEAN) - PLATOON 1,N,N,South,R,R2


In [6]:
# ,Info()To get the general information about the data and each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64186 entries, 0 to 64185
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Subject Age Group         64186 non-null  object 
 1   Subject ID                64186 non-null  int64  
 2   GO / SC Num               64186 non-null  float64
 3   Terry Stop ID             64186 non-null  int64  
 4   Stop Resolution           64186 non-null  object 
 5   Weapon Type               64186 non-null  object 
 6   Officer ID                64186 non-null  object 
 7   Officer YOB               64186 non-null  int64  
 8   Officer Gender            64186 non-null  object 
 9   Officer Race              64186 non-null  object 
 10  Subject Perceived Race    64186 non-null  object 
 11  Subject Perceived Gender  64186 non-null  object 
 12  Reported Date             64186 non-null  object 
 13  Reported Time             64186 non-null  object 
 14  Initia

2. Data Preparation

    2.1 Data Cleaning



In [17]:
# Drop for missing values

df_cleaned= df.dropna()

In [23]:
# Checking if missing values are removed
df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 63623 entries, 0 to 64185
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         63623 non-null  object
 1   Stop Resolution           63623 non-null  object
 2   Weapon Type               63623 non-null  object
 3   Officer ID                63623 non-null  object
 4   Officer YOB               63623 non-null  int64 
 5   Officer Gender            63623 non-null  object
 6   Officer Race              63623 non-null  object
 7   Subject Perceived Race    63623 non-null  object
 8   Subject Perceived Gender  63623 non-null  object
 9   Reported Date             63623 non-null  object
 10  Reported Time             63623 non-null  object
 11  Initial Call Type         63623 non-null  object
 12  Final Call Type           63623 non-null  object
 13  Call Type                 63623 non-null  object
 14  Officer Squad         

In [24]:
# Check for duplicates

df_cleaned.duplicated().sum()

113

In [25]:
# Drop duplicated rows
df_cleaned.drop_duplicates()

,Subject Age Group,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Reported Date,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,46 - 55,Field Contact,None,8544,1993,Female,Hispanic,White,Male,2018-07-30T00:00:00Z,47:00.0,-,-,-,NORTH PCT 2ND WATCH - B/N RELIEF,N,N,West,Q,Q2
1,18 - 25,Offense Report,None,8588,1986,Female,White,Black or African American,Male,2018-07-23T00:00:00Z,43:00.0,"OBS WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/D...","WEAPON, PERSON WITH - GUN",911,TRAINING - FIELD TRAINING SQUAD,N,Y,North,J,J1
2,26 - 35,Offense Report,None,4852,1953,Male,Asian,White,Male,2017-01-30T00:00:00Z,36:00.0,DISTURBANCE,THEFT - SHOPLIFT,911,NORTH PCT 2ND W - LINCOLN - PLATOON 1,N,N,North,L,L3
3,36 - 45,Field Contact,Knife/Cutting/Stabbing Instrument,7766,1984,Male,White,Native Hawaiian or Other Pacific Islander,Male,2023-05-02T00:00:00Z,25:58.0,OBS - DOWN - CHECK FOR PERSON DOWN,SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,WEST PCT 1ST W - QUEEN (DAVID) - PLATOON 1,N,Y,West,K,K3
4,Jan-17,Field Contact,-,8765,1992,Male,Two or More Races,Black or African American,Male,2023-03-31T00:00:00Z,53:15.0,MVC - HIT AND RUN,TRAFFIC - MV COLLISION INVESTIGATION,911,SOUTH PCT 3RD W - ROBERT (OCEAN) - PLATOON 1,N,N,South,R,R2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64181,26 - 35,Field Contact,-,8486,1992,Male,Asian,American Indian or Alaska Native,Female,2021-01-10T00:00:00Z,15:57.0,ASLT - DV,DV - DOMESTIC VIOL/ASLT (ARREST MANDATORY),"TELEPHONE OTHER, NOT 911",SOUTHWEST PCT 2ND W - FRANK - PLATOON 2,N,N,Southwest,F,F1
64182,26 - 35,Arrest,None,8337,1984,Male,Two or More Races,White,Male,2016-10-05T00:00:00Z,06:00.0,SUSPICIOUS STOP - OFFICER INITIATED ONVIEW,WARRANT SERVICES - MISDEMEANOR,ONVIEW,NORTH PCT 1ST W - BOY (JOHN) - PLATOON 1,N,Y,North,B,B2
64183,26 - 35,Arrest,None,8518,1987,Male,Asian,Unknown,Male,2017-11-23T00:00:00Z,25:00.0,OBS - DOWN - CHECK FOR PERSON DOWN,SUSPICIOUS CIRCUM. - SUSPICIOUS VEHICLE,911,TRAINING - FIELD TRAINING SQUAD,N,Y,North,L,L2
64184,36 - 45,Field Contact,-,8769,1995,Male,Declined to Answer,White,Male,2024-07-23T00:00:00Z,55:32.0,SUSPICIOUS STOP - OFFICER INITIATED ONVIEW,SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,EAST PCT 3RD W - EDWARD (CHARLIE) - PLATOON 1,N,N,East,E,E2
